<a href="https://colab.research.google.com/github/sayyedshoaib/Text-Summarization-using-LLM/blob/main/Text_Summarization_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!npm install localtunnel


In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:

%%writefile app.py
import streamlit as st
import wikipedia
import json
from langchain_core.prompts import ChatPromptTemplate
import time
from google.colab import userdata
import pandas as pd

from langchain_huggingface import HuggingFaceEndpoint

import os


os.environ["HF_TOKEN"] = <Your Hugging Face API token>

from langchain_huggingface import HuggingFaceEndpoint

sec_key = os.environ.get("HF_TOKEN")

repo_id="mistralai/Mistral-7B-Instruct-v0.3"
llm=HuggingFaceEndpoint(repo_id=repo_id,max_length=250,token=sec_key,temperature=0.4,timeout=300)


def get_data_of_scientist_wikipedia(url):
  name = url.replace("https://en.wikipedia.org/wiki/", "")
  text = wikipedia.page(name).content
  cleaned_text = ' '.join(text.split())
  return cleaned_text

def split_dom_content(dom_content, max_length=6000):
    return [
        dom_content[i : i + max_length] for i in range(0, len(dom_content), max_length)
    ]

template = (
    "You are tasked with extracting specific information from the following text content: {text_data}."
    "Please follow these instructions carefully: \n\n"
    "1. **Extract Information:** Only extract the information that directly matches the provided description: {parse_description}. "
    "2. **No Extra Content:** Do not include any additional text, comments, or explanations in your response. "
    "3. **Empty Response:** If no information matches the description, return an empty python dictionary ."
    "4. **Direct Data Only:** Your output should contain only the data that is explicitly requested, with no other text."
    "**Provide summary in about {words} words."
)
def parse_with_llm(text_chunks, parse_description,input_token):
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | llm

    parsed_results = []

    for i, chunk in enumerate(text_chunks, start=1):
        response = chain.invoke(
            {"text_data": chunk, "parse_description": parse_description,"words":input_token}
        )
        parsed_results.append(response)
        time.sleep(1)
    # print("--")
    return parsed_results

description = """extract the information from the given data and represent output in a json format having following key:
Title: The title of the Data .
Summary: A brief summary of the Data.
works: Scientific field in which that scientist worked (this field shoud be in string).

sample output:
{
"Title": "Albert Einstein",
"Summary": "Albert Einstein was a German-born theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics. His work is also known for its influence on the philosophy of science.",
"works": "theoretical physicist"
}
"""
def generate_summary(data,input_token):

  dom_chunks = split_dom_content(data)
  result = parse_with_llm(dom_chunks,description,input_token)

  text = result[0]
  newstring = ""
  f = 0
  for i in text:
    if(f == 1):
      newstring += i
    if(i == '{'):
      newstring += i
      f = 1
    elif(i == '}'):
      break

  text = newstring
  text = text.replace('\n', '').replace('`', '').strip()
  my_dict = json.loads(text)
  return my_dict

reference_summary = {"Aryabhata":"Aryabhata (476–550 CE) was an ancient Indian mathematician and astronomer. He is known for his work on algebra, trigonometry, and the approximation of π (pi). Aryabhata also proposed the heliocentric theory, suggesting that Earth rotates on its axis. His major work, the Aryabhatiya, covers mathematical and astronomical concepts, greatly influencing future scholars in India and the Islamic world.",
                     "Alexei Alexeyevich Abrikosov":"Alexei Abrikosov (1928–2017) was a Soviet-American theoretical physicist known for his work on superconductivity. He won the Nobel Prize in Physics in 2003 for discovering type-II superconductors, materials that can exhibit superconductivity in strong magnetic fields. His contributions significantly advanced the field of condensed matter physics.",
                     "Sir Arthur Stanley Eddington":"Arthur Eddington (1882–1944) was an English astronomer, physicist, and mathematician known for his work on the theory of general relativity. He led an expedition in 1919 that provided the first experimental confirmation of Einstein's theory. Eddington also made significant contributions to the understanding of stellar structures and astrophysics.",
                     "Boyd Wheeler Bartlett":"Boyd Wheeler Bartlett (1897–1965) was an American military officer, educator, and physicist. He served as head of the physics department at the United States Military Academy and made notable contributions to the field of electromagnetism. During World War II, he introduced a course in atomic physics, earning the Legion of Merit. He also co-authored research papers with physicist Arnold Sommerfeld.",
                     "Bertram Brockhouse":"Bertram Brockhouse was a Canadian physicist known for his pioneering work in the field of neutron scattering. He developed techniques that allowed for the study of atomic and molecular structures using neutrons, contributing significantly to condensed matter physics. In 1994, he was awarded the Nobel Prize in Physics for his innovative contributions.",
                     "Bimla Buti":"Bimla Buti is an Indian physicist renowned for her contributions to theoretical and experimental physics, particularly in the study of condensed matter and nanotechnology. She has made significant strides in understanding magnetic materials and has been instrumental in promoting science education in India. Buti has received numerous awards for her work and mentorship.",
                     "Carl David Anderson":"Carl David Anderson was an American physicist best known for discovering the positron in 1932, which provided crucial evidence for the theory of antimatter. He conducted his research using cloud chambers, and his work earned him the Nobel Prize in Physics in 1936. Anderson also contributed to studies in cosmic rays and high-energy particle physics.",
                     "Cornelis Bakker":"Cornelis Bakker was a Dutch physicist known for his contributions to plasma physics and nuclear fusion research. He played a significant role in advancing the understanding of plasma behavior in various conditions. Bakker's work has influenced both theoretical and experimental aspects of fusion energy, aiming to harness it as a viable energy source.",
                     "Charles Glover Barkla":"Charles Glover Barkla was a British physicist awarded the Nobel Prize in Physics in 1917 for his discovery of characteristic X-rays. His research significantly advanced the understanding of atomic structure and X-ray emissions. Barkla's work laid the groundwork for further studies in X-ray spectroscopy and contributed to the development of quantum mechanics.",
                     "Derek Abbott":"Derek Abbott is an Australian electrical engineer and physicist recognized for his research in areas such as nanotechnology, photonics, and energy harvesting. He has contributed significantly to the development of novel devices and systems, including solar energy technology. Abbott is also involved in education and mentoring, advocating for interdisciplinary approaches in science and engineering.",
                     "David Z. Albert":"David Z. Albert is an American theoretical physicist and philosopher known for his work in quantum mechanics and the philosophy of time. He has explored the implications of quantum theory on our understanding of reality, particularly concerning the nature of time and causality. Albert is also a prominent advocate for scientific literacy and public engagement with science.",
                     "David Awschalom":"David Awschalom is an American physicist recognized for his pioneering research in quantum information science and spintronics. He has made significant contributions to the understanding of quantum coherence and its applications in solid-state systems. Awschalom's work has implications for the development of quantum computing and advanced materials, enhancing the field's interdisciplinary nature.",
                     "Ernst Karl Abbe":"Ernst Karl Abbe was a German physicist and optician renowned for his contributions to optical theory and microscopy. He developed the Abbe sine condition, which improved the resolution of microscopes. Co-founder of the Zeiss company, Abbe also emphasized the importance of scientific research and education, significantly influencing optical instrument design and manufacturing.",
                     "Elephter Andronikashvili":"Elephter Andronikashvili was a Georgian physicist known for his work in the field of low-temperature physics and the properties of liquid helium. He made significant contributions to understanding superfluidity, particularly through his experiments that explored the behavior of helium at extremely low temperatures. His research has had lasting impacts on condensed matter physics.",
                     "Edward Victor Appleton":"Edward Victor Appleton was a British physicist awarded the Nobel Prize in Physics in 1947 for his pioneering work on the ionosphere. He developed methods to study its structure and properties using radio waves, significantly advancing the understanding of atmospheric physics. Appleton's research laid the groundwork for modern telecommunications and radio navigation systems.",
                     "Franz Aepinus":"Franz Aepinus was a German physicist and mathematician known for his early work in electromagnetism and electrostatics. He is recognized for formulating the concept of electric potential and contributing to the understanding of magnetic and electric fields. Aepinus's theories laid important groundwork for later developments in physics, particularly in the study of electromagnetism.",
                     "Felicie Albert":"Felicie Albert is an American physicist renowned for her work in high-energy physics and laser-driven particle acceleration. She has contributed significantly to advancing techniques for accelerating electrons and protons using laser technology, which has implications for future particle colliders. Albert is also an advocate for diversity in STEM fields and promotes education in science.",
                     "Frederick Grover":"Frederick Grover was an American physicist and engineer known for his contributions to the field of radio engineering and wave propagation. He is particularly recognized for developing the Grover's algorithm for improving communication systems. Grover's work laid the foundation for advancements in radio technology, influencing both military and civilian applications."}
def get_reference_summary(name):
  for key, value in reference_summary.items():
    if key == name:
      return value
  return ""


from rouge_score import rouge_scorer
# Create a scorer for ROUGE-1 and ROUGE-L
def calculate_rouge(reference_summary, candidate_summary):
  scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)
  r1score = scorer.score(candidate_summary, reference_summary)['rouge1']
  r1score_dict = r1score._asdict()
  r2score = scorer.score(candidate_summary, reference_summary)['rouge2']
  r2score_dict = r2score._asdict()
  rlscore = scorer.score(candidate_summary, reference_summary)['rougeL']
  rlscore_dict = rlscore._asdict()
  return [[r1score_dict['precision'],r1score_dict['recall'],r1score_dict['fmeasure']],
          [r2score_dict['precision'],r1score_dict['recall'],r1score_dict['fmeasure']],
          [rlscore_dict['precision'],rlscore_dict['recall'],rlscore_dict['fmeasure']]]



# URL input field at the top of the page
st.title("Research Summary Tool")
url = st.text_input("URL", placeholder="Enter URL here...")

left_column, right_column = st.columns(2)

# Left side of the app
with left_column:
    st.subheader("Research Information")

    row_names = ['Rouge 1', 'Rouge 2', 'Rouge L']
    col_names = ['Precision', 'Recall', 'Fmeasure']

# Right side of the app
with right_column:

    num_tokens = st.slider("No. of Tokens", min_value=30, max_value=80, step=5)
    submit = st.button("Submit")

# Process the inputs and update the outputs when the button is clicked
if submit:
    data = get_data_of_scientist_wikipedia(url)
    output = generate_summary(data, num_tokens)
    with left_column:

        scientist_name_output = st.text(f"Name of Scientist: {output['Title']}")
        research_topic_output = st.text(f"Research: {output['works']}")
        text_token_output = st.text(f"Text Tokens: {len(data.split())}")
        summary_token_output = st.text(f"Summary Tokens: {len(output['Summary'].split())}")


        ref_summary = get_reference_summary(output['Title'])
        if(len(ref_summary) != 0):
          rouge_score = calculate_rouge(ref_summary, output['Summary'])

          df = pd.DataFrame(rouge_score, index=row_names, columns=col_names)
          st.dataframe(df)

        else:
          rouge_score = [["-","-","-"],["-","-","-"],["-","-","-"]]
          df = pd.DataFrame(rouge_score,index=row_names, columns=col_names)


    with right_column:
        st.write(f"Summary: {output['Summary']}")
        st.write(num_tokens)



In [ ]:
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501